In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("C:\\Users\\PC\\Downloads\\lmq0411_2024-10-12.csv")

In [4]:
df.isnull().sum()

Type           0
Price          0
Area           0
Place          0
Link           0
Updated Day    0
dtype: int64

In [5]:
df = df.drop_duplicates()

In [6]:
df.head(7)

,Type,Price,Area,Place,Link,Updated Day
0,Căn hộ chung cư,"5,45 tỷ",85 m²,"Bình Thạnh, Hồ Chí Minh",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11
1,Nhà biệt thự liền kề,"11,9 tỷ",112 m²,"Văn Giang, Hưng Yên",https://batdongsan.com.vn/ban-nha-biet-thu-lie...,2024-10-11
2,Căn hộ chung cư,"4,5 tỷ","83,7 m²","Sơn Trà, Đà Nẵng",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11
3,Căn hộ chung cư,"1,52 tỷ","55,6 m²","Dĩ An, Bình Dương",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11
4,Căn hộ chung cư,"2,67 tỷ","69,7 m²","Dĩ An, Bình Dương",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11
5,Căn hộ chung cư,"13,5 tỷ","92,5 m²","Tây Hồ, Hà Nội",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11
6,Căn hộ chung cư,Giá thỏa thuận,70 m²,"Quận 7, Hồ Chí Minh",https://batdongsan.com.vn/ban-can-ho-chung-cu-...,2024-10-11


In [7]:
df['Updated Day'] = pd.to_datetime(df['Updated Day'])

In [8]:
df = df[df['Price'] != 'Giá thỏa thuận']

In [9]:
df = df[df['Price'] != "1.038,02 tỷ"]

In [10]:
condition = (df['Area'].str.count('\.') == 1) & (df['Area'].str.count(',') == 1)
df = df[~condition] 


In [11]:
condition = df['Area'].str.count('\.') > 1
# Loại bỏ các hàng thỏa mãn điều kiện
df = df[~condition]

In [12]:
# Hàm chuyển đổi
def convert_price(price):
    if pd.isna(price):
        return np.nan  
    if not isinstance(price, str):
        return float(price)  

    # Xử lý chuỗi giá
    price = price.replace(' nghìn', '')
    price = price.replace('/m²', '')
    price = price.replace('/tháng', '')
    price = price.replace(',', '.')  

    if ' tỷ' in price: 
        price = price.replace(' tỷ', '')
        price = float(price) * 1000  
    elif ' triệu' in price: 
        price = price.replace(' triệu', '')
        price = float(price)  
    return float(price)  

# Áp dụng hàm chuyển đổi cho cột 'Price'
df['Price_million_VND'] = df['Price'].apply(convert_price)
df = df.drop(columns=['Price'])


In [13]:
df['Area'] = df['Area'].str.replace(' m²', "").str.replace(',', '.').astype(float)

In [14]:
df[['District', 'City']] = df['Place'].str.split(',', expand=True)
df.drop(columns=['Place'], inplace=True)

In [15]:
df['Price_per_m2'] = round(df['Price_million_VND']/df['Area'], 2)

In [16]:
df[(df['District'] == 'Hoàn Kiếm') & (df['Price_million_VND'] > 9000)]

,Type,Area,Link,Updated Day,Price_million_VND,District,City,Price_per_m2
568,Nhà mặt phố,82.0,https://batdongsan.com.vn/ban-nha-mat-pho-pho-...,2024-10-11,98000.0,Hoàn Kiếm,Hà Nội,1195.12
880,Nhà mặt phố,158.0,https://batdongsan.com.vn/ban-nha-mat-pho-pho-...,2024-10-11,248000.0,Hoàn Kiếm,Hà Nội,1569.62
1505,Nhà mặt phố,64.0,https://batdongsan.com.vn/ban-nha-mat-pho-pho-...,2024-10-11,29500.0,Hoàn Kiếm,Hà Nội,460.94
2081,Nhà riêng,29.0,https://batdongsan.com.vn/ban-nha-rieng-duong-...,2024-10-11,29000.0,Hoàn Kiếm,Hà Nội,1000.00
2438,Nhà mặt phố,115.0,https://batdongsan.com.vn/ban-nha-mat-pho-pho-...,2024-10-11,58000.0,Hoàn Kiếm,Hà Nội,504.35
...,...,...,...,...,...,...,...,...
188918,Nhà riêng,45.0,https://batdongsan.com.vn/ban-nha-rieng-phuong...,2024-10-12,14500.0,Hoàn Kiếm,Hà Nội,322.22
189280,Nhà mặt phố,550.0,https://batdongsan.com.vn/ban-nha-mat-pho-duon...,2024-10-12,320000.0,Hoàn Kiếm,Hà Nội,581.82
189352,Nhà mặt phố,39.0,https://batdongsan.com.vn/ban-nha-mat-pho-duon...,2024-10-12,28000.0,Hoàn Kiếm,Hà Nội,717.95
189357,Nhà mặt phố,27.0,https://batdongsan.com.vn/ban-nha-mat-pho-pho-...,2024-10-12,24000.0,Hoàn Kiếm,Hà Nội,888.89


import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import count, when, col, split, udf, round 
from pyspark.sql.types import FloatType
from datetime import datetime

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
print(args)

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Đọc CSV từ S3
data_path = "s3://ai4e-ap-southeast-1-dev-s3-data-landing/lmq0411/raw/lmq0411_2024-10-12.csv"
df_csv = spark.read.csv(data_path, header=True, inferSchema=True)

# Làm phẳng dữ liệu
df = df_csv.select(
    col("Type").alias("Type"),
    col("Price").alias("Price_million_VND"),
    col("Area").alias("Area_m2"),
    col("Place").alias("Place"),
    col("Link").alias("Link"),
    col("Updated Day").alias("Updated_Day")
)

# Kiểm tra số lượng giá trị null
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Loại bỏ hàng có giá trị null
df = df.dropna()

# Loại bỏ các hàng trùng lặp
df = df.dropDuplicates()

# Chuẩn hóa cột Place và Type
def case_normalization(ne):
    return ' '.join([x.capitalize() for x in ne.split()])

normalize_place_udf = udf(case_normalization)

df = df.withColumn("Place", normalize_place_udf(df["Place"]))
df = df.withColumn("Type", normalize_place_udf(df["Type"]))

# Chuẩn hóa giá trị Price
def normalize_price(price):
    if "thỏa thuận" in price:
        return None
    price = price.replace(',', '.')
    if "tỷ" in price:
        return float(price.replace('tỷ', '').strip()) * 1_000
    elif "triệu" in price:
        return float(price.replace('triệu', '').strip())
    return float(price)

normalize_price_udf = udf(normalize_price, FloatType())
df = df.withColumn("Price_million_VND", normalize_price_udf(df["Price_million_VND"]))

# Tách cột Place thành District và City
df = df.withColumn("District", split(df["Place"], ",")[0]) \
       .withColumn("City", split(df["Place"], ",")[1])

# Xoá cột Place
df = df.drop("Place")

# Chuẩn hóa giá trị Area
def normalize_area(area):
    if area is None or area.strip() == "":
        return None
    area = area.replace(',', '.').replace(' m²', '').strip()
    try:
        return float(area)
    except ValueError:
        return None

normalize_area_udf = udf(normalize_area, FloatType())
df = df.withColumn("Area_m2", normalize_area_udf(df["Area_m2"]))

# Thêm cột price_per_m2
df = df.withColumn("price_per_m2", round(col("Price_million_VND") / col("Area_m2"), 2))

# Lưu kết quả dưới định dạng Parquet trong S3
destination_path = "s3://ai4e-ap-southeast-1-dev-s3-data-landing/lmq0411/bronze/processed_data.csv"
df.write.mode('overwrite').parquet(destination_path)

#### END JOB IMPLEMENTATION -> ETLs END
job.commit()